<a href="https://colab.research.google.com/github/dobrotvorn/DataCon23/blob/main/Hackathon_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Описание задания


Задание заключается в построении модели прогнозирования зоны ингибирования для антибиотиков и наночастиц серебра при взаимодействии с бактреиями.

## Описание данных
data.csv

Для всех образцов в датасете использовались наночастицы серебра Ag

---



*   Bacteria: бактериальная клетка-мишень
* NP_Synthesis: тип синтеза наночастиц
* drug: название лекарства
* drug class: класс лекарства
* Drug dose: доза препарата
* NP_concentration: концентрация НЧ
* NP_size: размер НЧ
* shape: форма НЧ
* method: метод определения антимикробной активности
* ZOI_drug, ZOI_NP, ZOI_drug_NP: зона ингибирования препарата, НЧ и их комбинации
* Fold increase in antibacterial activity (%): кратное увеличение
* антибактериальной активности (%)
* дзета-потенциал;
* MDR: множественная лекарственная устойчивость резистентен ли обычный препарат к целевому патогену

---

bacterial_descriptors.csv


* Tax_id: id бактерии в базе данных NCBI
* Bacteria: название бактерии
* ingdom: царство бактерии
* subkingdom: подцарство бактерии
* clade: характеристика бактерии с точки зрения родственных взаимоотношений между таксономическими группами
* phylum: филум бактерии
* class: класс бактерии
* order: порядок бактерии
* family: семейство бактерии
* genus: род бактерии
* species: вид бактерии
* gram: результат реакции окрашивания по Грамму для бактерии
* min_Incub_period, avg_Incub_period, max_Incub_period h: характеристики инкубационного периода бактерии
* growth_temp, C: температура роста бактерии
* biosafety_level: уровень опасности бактерии
* isolated_from: источник


---

drug_descriptors.csv
* drug: название лекарства
* chemID: id лекарства в базе данных CHEMBL
* prefered_name: название лекарства
* smiles: химическая формула лекарства

## Загрузка данных и библиотек

###Импортируем и скачиваем библиотеки

In [ ]:
!pip install gdown -- quiet
!pip install pymatgen
!pip install rdkit
!pip install fancyimpute --quiet
!pip install shap
!pip install livelossplot --quiet

from IPython.display import clear_output
import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import plotly

from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin


import torch

import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import plotly
import scipy.stats as stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import KNNImputer
from IPython.display import display, HTML
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from fancyimpute import KNN
import pymatgen.core as mg
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import SimilarityMaps
from zipfile import ZipFile

%matplotlib inline
import pandas as pd
from IPython.display import clear_output
import numpy as np
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import  MinMaxScaler,PolynomialFeatures,StandardScaler
from sklearn.impute import  KNNImputer, SimpleImputer, IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, KBinsDiscretizer # говорят, он хорош для выбрасов
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import StratifiedKFold #так как у нас несбалансированные таргеты (много небольших цен и мало больших), возможно, стоит так стратифицировать.
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor, VotingRegressor
from sklearn.linear_model import SGDRegressor, RANSACRegressor, TheilSenRegressor, HuberRegressor, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import Binarizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from time import sleep
from matplotlib import pyplot as plt
import numpy as np
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot



from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import KNNImputer
from IPython.display import display, HTML
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from fancyimpute import KNN
from re import A
import json
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
clear_output()

In [ ]:
# Тут захардкодим статичные переменные
global ordinal_encoder
global text_columns
global path2firstDb
global path2secondDb
global numerics
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
path2data = '/content/data.csv' if 'google.colab' in sys.modules else 'data.csv'
path2bac_descr = '/content/bacterial_descriptors.csv' if 'google.colab' in sys.modules else 'bacterial_descriptors.csv'
path2drug_descr = '/content/drug_descriptors.csv' if 'google.colab' in sys.modules else 'drug_descriptors.csv'

### Скачиваем данные

In [ ]:
url1 = 'https://raw.githubusercontent.com/dataconHack/hackathon/main/data.csv'
url2 = 'https://raw.githubusercontent.com/dataconHack/hackathon/main/bacterial_descriptors.csv'
url3 = 'https://raw.githubusercontent.com/dataconHack/hackathon/main/drug_descriptors.csv'
output1 = 'data.csv'
output2 = 'bacterial_descriptors.csv'
output3 = 'drug_descriptors.csv'
gdown.download(url1, output1, quiet=True)
gdown.download(url2, output2, quiet=True)
gdown.download(url3, output3, quiet=True)
clear_output()

## Обработка данных

In [ ]:
fdb = pd.read_csv(path2data)
fdb_bak = pd.read_csv(path2bac_descr)
fdb_drug = pd.read_csv(path2drug_descr)

In [ ]:
# Добавляем дескрипторы из RDKit
descript = ['MolLogP', 'NumValenceElectrons', 'NumHeteroatoms', 'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'TPSA', 'LabuteASA']
def get_descriptors_values(desc, mol):
    descriptors_values = []
    for descr in descript:
        descriptors_values.append(Descriptors.CalcMolDescriptors(mol)[descr])
    return descriptors_values
filled_descriptors_list = []
for item in fdb_drug['smiles']:
  mol = Chem.MolFromSmiles(f'{item}')
  descriptors_with_value = get_descriptors_values(descript, mol)
  filled_descriptors_list.append(descriptors_with_value)
fdb_drug_desc = pd.DataFrame(filled_descriptors_list, columns=descript)
fdb_drug = fdb_drug.join(fdb_drug_desc, how='outer')

In [ ]:
# Сольём три табличка в одну ( null не заполняли - лучше мб заполнить сразу же до мерджа)
db =  fdb.merge(fdb_bak, how='left', left_on='Bacteria', right_on='Bacteria').merge(fdb_drug, how='left', left_on='Drug', right_on='drug').drop(['Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 0_y', 'Drug'], axis=1)

In [ ]:
# Заменим остальные пропуски в данных на None, потому что сильных зависимостей не обнаружено (за исключением Surface_Charge, но там пришли к мнению, что нейтральный - это 0)
none_mapper = { '-': np.nan, 'None': np.nan, None: np.nan, 'nan': np.nan, pd.NA : np.nan}
db.replace(none_mapper, inplace=True)

### Преобразуем некоторые данные, заменим типы

In [ ]:
# Преобразует ZOI типа "32+", "17+2" в числа
def ZOI_transform(x):
  a = None
  try:
    a = eval(x)
  except:
    if type(x) == float or type(x) == int:
      return a
    a = x.split('+')[0]
  return a

# Удалим несколько строк с царством грибов
db = db.loc[db['kingdom'] != 'Fungi']

# Убирает странные концентрации НЧ
def NP_conc_transform(x):
  a = None
  try:
    a = float(x)
  except:
    return np.nan

# Преобразуем данные ZOI и концентрации НЧ
db['ZOI_drug'] = db['ZOI_drug'].apply(lambda x: ZOI_transform(x) if x else np.nan)
db['ZOI_NP'] = db['ZOI_NP'].apply(lambda x: ZOI_transform(x) if x else np.nan)
db['ZOI_drug_NP'] = db['ZOI_drug_NP'].apply(lambda x: ZOI_transform(x) if x else np.nan)
db['NP_concentration'] = db['NP_concentration'].apply(lambda x: ZOI_transform(x) if x else np.nan)
# Меняем тип данных на float
db = db.astype({'ZOI_drug': np.float, 'ZOI_NP': np.float, 'ZOI_drug_NP': np.float, 'NP_concentration': np.float})

In [1]:
# Заменим выбросы в бактериях на данные из исследований
db_without_nulls_in_target[(db_without_nulls_in_target['Bacteria'] == 'Acinetobacter baumanii') & (db_without_nulls_in_target['ZOI_NP'] == 1000)] = 13.6
db_without_nulls_in_target[(db_without_nulls_in_target['Bacteria'] == 'Escherichia coli') & (db_without_nulls_in_target['ZOI_NP'] == 1000)] = 11
db_without_nulls_in_target[(db_without_nulls_in_target['Bacteria'] == 'Proteus sp.') & (db_without_nulls_in_target['ZOI_NP'] == 1000)] = 17.8
db_without_nulls_in_target[(db_without_nulls_in_target['Bacteria'] == 'Pseudomonas aeruginosa') & (db_without_nulls_in_target['ZOI_NP'] == 1000)] = 16
db_without_nulls_in_target[(db_without_nulls_in_target['Bacteria'] == 'Klebsiella sp') & (db_without_nulls_in_target['ZOI_NP'] == 1000)] = 17.8

NameError: ignored

 ### Проверка

In [ ]:
#поиск пустых ячеек
cols = db.columns
# желтый - пропущенные данные, синий - не пропущенные
colours = ['#000099', '#ffff00']
sns.heatmap(db[cols].isnull(), cmap=sns.color_palette(colours))
sns.set (font_scale=0.5)
# процентный список пропущенных данных
for col in db.columns:
    pct_missing = np.mean(db[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
#Корреляционная карта
corr = db.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(12, 10))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, cmap=cmap, linewidths=.5, mask=mask,annot=True, fmt='.1f')